# Download

In [ ]:
WORKING_ENV = 'LOCAL' # Can be LABS, COLAB or PAPERSPACE
assert WORKING_ENV in ['LABS', 'COLAB', 'LOCAL']

import sys
if WORKING_ENV == 'COLAB':
  from google.colab import drive
  drive.mount('/content/drive/')
  !pip install medmnist
  !pip install torch
  !pip install gputil
  !pip install psutil
  !pip install humanize
  ROOT = "/content/drive/MyDrive/ColabNotebooks/med-contrastive-project/"
  sys.path.append(ROOT + "./moco/")
  !nvidia-smi
elif WORKING_ENV == 'LABS':
  ROOT = "/vol/bitbucket/sx119/Contrastive-Medical-Image-Classification/"
else:
  ROOT = "/Users/xushitong/Contrastive-Medical-Image-Classification/"

# Import

In [ ]:
import medmnist

import argparse
import builtins
import math
import os
import random
import shutil
import time
import tqdm
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
# import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

mem_report()

# Hyperparameters

# Dataset

In [ ]:
# Data loading code
# traindir = os.path.join(args.data, 'train')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
augmentation = [
    transforms.RandomResizedCrop(224, scale=(0.2, 1.)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]

# # legacy split dataset operation, should load splitted data instead
# train_dataset = medmnist.PathMNIST("train", download=False, root=args.data, 
#                                    transform=loader.TwoCropsTransform(transforms.Compose(augmentation)))
# val_dataset = medmnist.PathMNIST("val", download=False, root=args.data, 
#                                    transform=loader.TwoCropsTransform(transforms.Compose(augmentation)))

# pretrain_len = int(len(train_dataset) * TRAIN_SET_RATIO)
# pretrain_set, pretrain_val_set = torch.utils.data.random_split(train_dataset, [pretrain_len, len(train_dataset) - pretrain_len])
# torch.save(pretrain_set, "pretrain_set.data")
# torch.save(pretrain_val_set, "pretrain_val_set.data")

# # proper dataset loading, by loading pre-splitted data
pretrain_set = torch.load(args.data + "/pretrain_set.data")
pretrain_val_set = torch.load(args.data + "/pretrain_val_set.data")
val_dataset = medmnist.PathMNIST("val", download=False, root=args.data, 
                                   transform=loader.TwoCropsTransform(transforms.Compose(augmentation)))

pretrain_loader = torch.utils.data.DataLoader(
    pretrain_set, batch_size=args.batch_size, shuffle=True, 
    pin_memory=True, drop_last=True)
pretrain_val_loader = torch.utils.data.DataLoader(
    pretrain_val_set, batch_size=2 * args.batch_size, shuffle=False, 
    pin_memory=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=2 * args.batch_size, shuffle=False, 
    pin_memory=True, drop_last=True)

print(f"pretrain size: {len(pretrain_set)}\npretrain validation size: {len(pretrain_val_set)}\nvalidation size, {len(val_dataset)}")


In [ ]:
train_dataset = medmnist.PathMNIST("train", download=False, root=args.data)

pretrain_len = int(len(train_dataset) * TRAIN_SET_RATIO)
pretrain_set, pretrain_val_set = torch.utils.data.random_split(train_dataset, [pretrain_len, len(train_dataset) - pretrain_len])
torch.save(pretrain_set, "pretrain_set.data")
torch.save(pretrain_val_set, "pretrain_val_set.data")